#### 감성분석 --> 머신러닝
    - 데이터셋 : 전처리
    - BoW 모델
        - 단어를 특성 벡터로 변환
        - tf-idf 단어 적합성 평가
        - 텍스트 데이터 정제
        - 문서를 토큰으로 나누기
    - LogisticRegression

In [42]:
import pandas as pd
from glob import glob
file_lists = glob('/Users/joonseok/Desktop/AI/VS Code/Analysis/aclImdb/train/neg/*.txt')
pd_lists = []
for file_path in file_lists[:]:   
    with open(file_path, 'r', encoding='utf-8') as f:
        data = {
            'review' : f.read(),
            'target' : 0
        }
        df = pd.DataFrame([data])
        pd_lists.append(df)
train_neg_df = pd.concat(pd_lists, ignore_index=True)        
train_neg_df.head()

,review,target
0,Working with one of the best Shakespeare sourc...,0
1,"Well...tremors I, the original started off in ...",0
2,Ouch! This one was a bit painful to sit throug...,0
3,"I've seen some crappy movies in my life, but t...",0
4,"""Carriers"" follows the exploits of two guys an...",0


In [43]:
# positive 동일하게 train_pos_df
# train_df = pd.concat([train_neg_df, train_pos_df])
# movie_data.csv로 저장


file_lists = glob('/Users/joonseok/Desktop/AI/VS Code/Analysis/aclImdb/train/pos/*.txt')
pd_lists = []
for file_path in file_lists[:]:   
    with open(file_path, 'r', encoding='utf-8') as f:
        data = {
            'review' : f.read(),
            'target' : 1
        }
        df = pd.DataFrame([data])
        pd_lists.append(df)
train_pos_df = pd.concat(pd_lists, ignore_index=True)        
train_pos_df.head()

,review,target
0,For a movie that gets no respect there sure ar...,1
1,Bizarre horror movie filled with famous faces ...,1
2,"A solid, if unremarkable film. Matthau, as Ein...",1
3,It's a strange feeling to sit alone in a theat...,1
4,"You probably all already know this by now, but...",1


In [44]:
train_df = pd.concat([train_pos_df, train_neg_df], ignore_index=True)
train_df.head()

,review,target
0,For a movie that gets no respect there sure ar...,1
1,Bizarre horror movie filled with famous faces ...,1
2,"A solid, if unremarkable film. Matthau, as Ein...",1
3,It's a strange feeling to sit alone in a theat...,1
4,"You probably all already know this by now, but...",1


In [45]:
train_df.to_csv('movie_data.csv', index=False, encoding = 'utf-8')

In [46]:
df = pd.read_csv('movie_data.csv')
df.head()

,review,target
0,For a movie that gets no respect there sure ar...,1
1,Bizarre horror movie filled with famous faces ...,1
2,"A solid, if unremarkable film. Matthau, as Ein...",1
3,It's a strange feeling to sit alone in a theat...,1
4,"You probably all already know this by now, but...",1


In [47]:
# BoW(Bag of Words) 모델
# 문자를 숫자벡터
# 단어의 등장횟수 카운트
# 전체 훈련데이터에서 모든 고유한 단어(토큰)로 어휘 사전
# 각 문서(리뷰데이터)를 사전을 기준으로 벡터화  N번째 단어가 문서에서 3번 나오면 벡터의 N번째 값이 3이 된다.
# 문서1: "나는 영화가 좋다"
# 문서2 : "나는 영화가 싫다"
# 사전 : {'나는' : 0, '영화가' : 1, '좋다' : 2, '싫다' : 3}
# 벡터화는 사전의 크기만큼 모든 문장의 길이를 동일하게
# 문서 1 벡터 : [0, 1, 2] --> [1, 1, 1, 0]
# 문서 2 벡터 : [0, 1, 3] --> [1, 1, 0, 1]

In [48]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array([
        "The sun is shining",
        "The weather is sweet"
])
bag = count.fit_transform(docs)

In [49]:
count.vocabulary_

{'the': 4, 'sun': 2, 'is': 0, 'shining': 1, 'weather': 5, 'sweet': 3}

In [50]:
# 문서 d에 등장한 단어 t의 횟수를 tf(t, d)
# Bow를 보완하면서 좀 더 정교한 텍스트 벡터화 방식  TF-IDF(Terms Frequency - Inverse Document Frequency)
# TF : 특정 문서에서 자주 등장하는 단어
# IDF : 전체 문서에 드물게 등장하는 단어
# 특정문서에서 자주 등장하지만 전체 문장에서 드물게 등장하는 단어에 높은 가중치를 부여 - 그 문장을 잘 대표하는 핵심 단어를 찾는다
# TF(t, d)  단어 t가 문장 d에 나타난 횟수 / 문서 d의 모든 단어 수
# IDF(t, D) : log(총 문서 수 |D| / 단어 t를 포함한 문서의 수 df(t)) --> log를 씌워 단어의 희귀성을 과하게 반영하지 않도록 스케일링 
# 분모에 +1(사이킷런의 경우) : 분모가 0이 되는 것을 방지
# log(1+ |D|) / 1+df(f))
# TF-IDF(t, d, D) = TF(t, d) X IDF(t, D) 

# "나는"
# TF : 리뷰에 3번 나옴 (높음)
# IDF : 전체 10,000개 리뷰 중에 9,000개 나옴(매우 낮음)
# tf-idf --> 높음 X 매우 낮음 = 낮음 (중요도가 낮음)

# "명작" TF : 리뷰에 2번 나옴 (높음)
# IDF : 전체 10,000개 리뷰 중에 50개 나옴(매우 높음)
# tf-idf --> 높음 X 매우 높음 = 높음 (핵심단어)

In [51]:
# 데이터 정제 : html tag와 같은 불필요한 string이 보임. 특수기호 등
df.review[0]

'For a movie that gets no respect there sure are a lot of memorable quotes listed for this gem. Imagine a movie where Joe Piscopo is actually funny! Maureen Stapleton is a scene stealer. The Moroni character is an absolute scream. Watch for Alan "The Skipper" Hale jr. as a police Sgt.'

In [52]:
# 데이터 정재... html tag 와 같은 불필요한 string이 보임... 특수기호 기타등등.  < - ' 
import re
def preprocessor(s):
    # 1. 영문, 공백, ., , 만 남기기
    clean = re.sub(r'[^A-Za-z\s.,]+', '', s)
    # 2. 연속된 마침표(...)를 마침표 하나로
    clean = re.sub(r'\.{2,}', '.', clean)
    # 3. 연속된 공백 정리
    clean = re.sub(r'\s+', ' ', clean).strip()
    return clean

In [53]:
df['review'] = df.review.apply(preprocessor)

In [54]:
# 문서를 토큰으로 나누기
# %pip install nltk

In [55]:
from nltk.stem.porter import PorterStemmer

def tokenizer(text):
    return text.split()

porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

# 어간추출 Stemming : 단어의 접미사(-s, -es, -ing, -ed 등)를 강제로 제거해서 단어의 원형을 찾는 과정
tokenizer(df.review[0][:100])


['For',
 'a',
 'movie',
 'that',
 'gets',
 'no',
 'respect',
 'there',
 'sure',
 'are',
 'a',
 'lot',
 'of',
 'memorable',
 'quotes',
 'listed',
 'for',
 'this',
 'gem.',
 'Imagi']

In [56]:
tokenizer_porter('runners like running')

['runner', 'like', 'run']

In [63]:
# 불용어
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')  # 불용어 사전 다운로드
stops = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/joonseok/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [59]:
df.head() # 정규식을 이용한 전처리완료

,review,target
0,For a movie that gets no respect there sure ar...,1
1,Bizarre horror movie filled with famous faces ...,1
2,"A solid, if unremarkable film. Matthau, as Ein...",1
3,Its a strange feeling to sit alone in a theate...,1
4,"You probably all already know this by now, but...",1


In [75]:
from sklearn.model_selection import train_test_split
X = df.review
y = df.target
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=42)


In [76]:
x_train

7522     Sandler is amazing again. I have already becom...
17493    This is crap.utter crap. I cannot believe any ...
12260    Okay, Ill admit itI am a goofball and I occasi...
1275     So, Fox pulled the plug midway through a drama...
8004     Though this series only ran a season, it has s...
                               ...                        
10080    Our Song is a marvelous example of passionate,...
7973     If you have not heard of this film from Walt D...
22968    Words cannot describe how utterly abysmal this...
17663    When the film began, I was shocked to see it w...
10894    This was playing at our theater in Amsterdam a...
Name: review, Length: 20000, dtype: object

In [77]:
x_train.ndim

1

In [64]:
# 어간추출(postStemmer) --> stopwor에 포함된 단어제거
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split() if word not in stops]

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [73]:
tfidf = TfidfVectorizer(
    tokenizer = tokenizer_porter,
    ngram_range = (1, 1) # (1, 11) 유니그램(unigram, 단일단어)만 사룡
)
pipeline = Pipeline([
    ('tfidf', tfidf),
    ('clf', LogisticRegression())
])

In [78]:
pipeline.fit(x_train, y_train)

/opt/anaconda3/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenizer_porter at 0x348403420>)),
                ('clf', LogisticRegression())])

In [79]:
print(x_test.to_numpy()[0])
print(y_test.to_numpy()[0])

I watched this movie years ago in company with my best female friend. I got my judgment teeth pulled out so I didnt feel very good.br br I ended up liking it big time. Its a hard watch if you take in account that it deals with friendship, unwanted betrayal, power, money, drug traffic, and the extreme hard situation that deals with living in a foreign jail.br br The acting is on its prime level. Two of the women that I lust the most star and thats a good thing. Claire Danes is as cute and charming as always while Kate Beckinsale is extremely hot and delivers a fine performance. Bill Pullman is also great and demonstrates his histrionic qualities.br br There are many plot twists to dig from and make it an interesting visual experience. Plus it shows the difficult times at Thailand.br br This is an underrated movie. Not many films like this one have come up in recent history. It should make you reflex about many things.
1


In [80]:
pipeline.score(x_test,y_test)

0.8802

In [81]:
# 원본데이터로드..
# 토크나이져 함수를 정의
    # 텍스트 전처리
    # 공백을 기준으로 단어단위로 분리
    # 영어는 전부 소문자로 변환
    # 어간 추출
    # 불용어 제거
# TFIDF를 정의
    # 토크나이져 매개변수 = 토크나이져 함수
    # ngram  (1,1)
# 파이프라인으로 tfidf, 머신러닝
# 파이프라이으로 학습
# 파이프라인으로 평가( classification_report)
# 과적합여부 확인

# train 폴더에 있는데이터로 학습 - 적당한 크기로
# test 폴더에 있는 문장으로 평가

In [82]:
import re
porter = PorterStemmer()
def custom_tokenizer(text):
    # 전처리
    # 1. 영문, 공백, ., , 만 남기기
    clean = re.sub(r'[^A-Za-z\s.,]+', '', text)
    # 2. 연속된 마침표(...)를 마침표 하나로
    clean = re.sub(r'\.{2,}', '.', clean)
    # 3. 연속된 공백 정리
    clean = re.sub(r'\s+', ' ', clean).strip()
    # 단어분리-어간분리-불용어제거
    return [porter.stem(word) for word in clean.split() if word not in stops]
tfidf = TfidfVectorizer(
    tokenizer=custom_tokenizer,
    ngram_range=(1,1),
    token_pattern=None
)
pipeline = Pipeline([
    ('tfid',tfidf),
    ('clf',LogisticRegression())
])

In [85]:
pipeline.fit(x_train,y_train)

Pipeline(steps=[('tfid',
                 TfidfVectorizer(token_pattern=None,
                                 tokenizer=<function custom_tokenizer at 0x34a3f4c20>)),
                ('clf', LogisticRegression())])

In [86]:
pipeline.score(x_test,y_test)

0.8802

In [87]:
# 원본데이터로드..
# 토크나이져 함수를 정의
    # 텍스트 전처리
    # 공백을 기준으로 단어단위로 분리
    # 영어는 전부 소문자로 변환
    # 어간 추출
    # 불용어 제거
# TFIDF를 정의
    # 토크나이져 매개변수 = 토크나이져 함수
    # ngram  (1,1)
# 파이프라인으로 tfidf, 머신러닝
# 파이프라이으로 학습
# 파이프라인으로 평가( classification_report)
# 과적합여부 확인

# train 폴더에 있는데이터로 학습 - 적당한 크기로
# test 폴더에 있는 문장으로 평가

In [88]:
from glob import glob
import numpy as np
from tqdm import tqdm

def get_data(pattern, neg=True,to = None):
    documents = []
    if to is not None:
        for path in tqdm(glob(pattern)[ : to]):
            with open(path, 'r', encoding='utf-8') as f:
                documents.append(  (np.array(f.read()),  0 if neg else 1)   )
    else:
        for path in tqdm(glob(pattern)):
            with open(path, 'r', encoding='utf-8') as f:
                documents.append(  (np.array(f.read()),  0 if neg else 1)   )
    return documents

# "../data/movie/train/neg/*.txt" 

In [89]:
train_neg_lists = get_data("../data/movie/train/neg/*.txt",True,5000)
train_pos_lists = get_data("../data/movie/train/pos/*.txt",False,5000)
test_neg_lists = get_data("../data/movie/test/neg/*.txt",True,5000)
test_pos_lists = get_data("../data/movie/train/pos/*.txt",False,5000)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [90]:
train_lists = train_neg_lists + train_pos_lists
test_lists = test_neg_lists + test_pos_lists
len(train_lists), len(test_lists)

(0, 0)

In [91]:
import re
porter = PorterStemmer()
def custom_tokenizer(text):
    # 전처리
    # 1. 영문, 공백, ., , 만 남기기
    clean = re.sub(r'[^A-Za-z\s.,]+', '', text)
    # 2. 연속된 마침표(...)를 마침표 하나로
    clean = re.sub(r'\.{2,}', '.', clean)
    # 3. 연속된 공백 정리
    clean = re.sub(r'\s+', ' ', clean).strip()
    # 단어분리-어간분리-불용어제거
    return [porter.stem(word) for word in clean.split() if word not in stops]
tfidf = TfidfVectorizer(
    tokenizer=custom_tokenizer,
    ngram_range=(1,1),
    token_pattern=None
)
pipeline = Pipeline([
    ('tfid',tfidf),
    ('clf',LogisticRegression())
])

In [ ]:
x = np.array([str(x.item()) for x,_ in train_lists])
y = np.array([ y for _,y in train_lists])

In [ ]:
pipeline.fit(x,y)

In [ ]:
x_test = np.array([str(x.item()) for x,_ in test_lists])
y_test = np.array([ y for _,y in test_lists])
pipeline.score(x_test, y_test)

In [ ]:
from sklearn.metrics import classification_report
print( classification_report(y_test, pipeline.predict(x_test)) )